# Scraping: récupération de la page d'accueil du journal Le Soir

Dans ce notebook, nous créons un robot qui va ouvrir la page d'accueil du site du journal [Le Soir](https://www.lesoir.be/) et récupérer le titre de tous les articles du jour et les stocker dans un fichier csv.

## Imports

In [9]:
import os
import certifi
#import re
#import time
import requests
from bs4 import BeautifulSoup
#import pandas as pd
from pathlib import Path

## Telechargement de la page 



In [24]:
#Le site de cours "https://max.de.wilde.web.ulb.be/camille/" présente un certificat SSL non reconnu par Python.
# La vérification a donc été désactivée via `verify=False` pour permettre le téléchargement de la page.


headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

# base_url (équivalent de root_url)
base_url = "https://max.de.wilde.web.ulb.be/camille/"

response = requests.get(base_url, headers=headers, verify=False)

print("Page téléchargée" if response.status_code == 200 else f" Erreur {response.status_code}")

Page téléchargée


c:\Users\Ing Armel Fopa\Desktop\cours M2\Bloc 1 2026\STIC-B545 - Traitement automatique de corpus - 202526\PROJET\Projet_TAC_1\PROJET_TAC_1_venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'max.de.wilde.web.ulb.be'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Extractions des liens PDF

In [25]:
from urllib.parse import urljoin

# on parse la page
soup = BeautifulSoup(response.text, 'html.parser')

# liste où on collectera les URLs / noms de fichiers PDF
pdf_files = []

for link in soup.find_all("a", href=True):
    href = link.get("href").strip()

    full_url = urljoin(base_url, href)

    if ".pdf" in full_url.lower():

        title = link.text.strip() or full_url.split("/")[-1]
        pdf_files.append([full_url, title])

print(f"{len(pdf_files)} liens trouvés.")
pdf_files[:10]

51 liens trouvés.


[['https://max.de.wilde.web.ulb.be/camille/KB_JB230_1892-08-07_01-0003.pdf',
  'KB_JB230_1892-08-07_01-0003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB427_1920-01-10_01-00004.pdf',
  'KB_JB427_1920-01-10_01-00004.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB555_1836-02-08_01-00002.pdf',
  'KB_JB555_1836-02-08_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB638_1860-05-21_01-00002.pdf',
  'KB_JB638_1860-05-21_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB773_1918-11-30_01-00002.pdf',
  'KB_JB773_1918-11-30_01-00002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB838_1887-12-28_01-00003.pdf',
  'KB_JB838_1887-12-28_01-00003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB230_1903-10-16_01-0002.pdf',
  'KB_JB230_1903-10-16_01-0002.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB427_1933-01-04_01-00003.pdf',
  'KB_JB427_1933-01-04_01-00003.pdf'],
 ['https://max.de.wilde.web.ulb.be/camille/KB_JB555_1899-01-19_01-00

Preparation du dossier de destination

In [44]:
data_dir = Path("../data/camille_pdf")
data_dir.mkdir(parents=True, exist_ok=True)

print(f"Dossier de destination : {data_dir}")
print(f"{len(pdf_files)} fichiers PDF trouvés sur la page CAMille.\n")

n = 5
for i, link in enumerate (pdf_files[:n], start=1):
     print(i, link)

Dossier de destination : ..\data\camille_pdf
51 fichiers PDF trouvés sur la page CAMille.

1 ['https://max.de.wilde.web.ulb.be/camille/KB_JB230_1892-08-07_01-0003.pdf', 'KB_JB230_1892-08-07_01-0003.pdf']
2 ['https://max.de.wilde.web.ulb.be/camille/KB_JB427_1920-01-10_01-00004.pdf', 'KB_JB427_1920-01-10_01-00004.pdf']
3 ['https://max.de.wilde.web.ulb.be/camille/KB_JB555_1836-02-08_01-00002.pdf', 'KB_JB555_1836-02-08_01-00002.pdf']
4 ['https://max.de.wilde.web.ulb.be/camille/KB_JB638_1860-05-21_01-00002.pdf', 'KB_JB638_1860-05-21_01-00002.pdf']
5 ['https://max.de.wilde.web.ulb.be/camille/KB_JB773_1918-11-30_01-00002.pdf', 'KB_JB773_1918-11-30_01-00002.pdf']


Téléchargement des fichiers PDF

In [48]:
total = len(pdf_files)
failures = []

for i, (pdf_url, title) in enumerate(pdf_files, start=1):
    filename = pdf_url.split("/")[-1]
    dest = data_dir / filename

    # Si le fichier est déjà présent et non vide, on le saute
    if dest.exists() and dest.stat().st_size > 0:
        print(f"[{i}/{total}] Déjà présent : {filename}")
        continue

    try:
        with requests.get(pdf_url, headers=headers, verify=False, stream=True, timeout=60) as r:
            r.raise_for_status()
            with open(dest, "wb") as f:
                for chunk in r.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
        if dest.stat().st_size > 0:
            print(f"[{i}/{total}] Téléchargé : {filename}")
        else:
            failures.append((pdf_url, "Taille 0"))
            print(f"[{i}/{total}] Fichier vide : {filename}")
    except Exception as e:
        failures.append((pdf_url, str(e)))
        print(f"[{i}/{total}] Erreur sur {filename} : {e}")

[1/51] Déjà présent : KB_JB230_1892-08-07_01-0003.pdf
[2/51] Déjà présent : KB_JB427_1920-01-10_01-00004.pdf
[3/51] Déjà présent : KB_JB555_1836-02-08_01-00002.pdf
[4/51] Déjà présent : KB_JB638_1860-05-21_01-00002.pdf
[5/51] Déjà présent : KB_JB773_1918-11-30_01-00002.pdf
[6/51] Déjà présent : KB_JB838_1887-12-28_01-00003.pdf
[7/51] Déjà présent : KB_JB230_1903-10-16_01-0002.pdf
[8/51] Déjà présent : KB_JB427_1933-01-04_01-00003.pdf
[9/51] Déjà présent : KB_JB555_1899-01-19_01-00003.pdf
[10/51] Déjà présent : KB_JB638_1902-12-20_01-00002.pdf
[11/51] Déjà présent : KB_JB773_1933-10-07_01-00007.pdf
[12/51] Déjà présent : KB_JB838_1911-08-03_01-00006.pdf
[13/51] Déjà présent : KB_JB230_1913-07-05_01-0001.pdf
[14/51] Déjà présent : KB_JB427_1949-07-18_01-00008.pdf
[15/51] Déjà présent : KB_JB555_1940-03-01_01-00004.pdf
[16/51] Déjà présent : KB_JB638_1946-07-18_01-00003.pdf
[17/51] Déjà présent : KB_JB773_1950-07-22_01-00010.pdf
[18/51] Déjà présent : KB_JB838_1943-09-04_01-00002.pdf
[19/

Verification du nombre de fichiers telechargés

In [50]:
print("\n--- RÉSUMÉ ---")
print(f"Attendus : {total} | Téléchargés : {len(list(data_dir.glob('*.pdf')))} | Échecs : {len(failures)}")


--- RÉSUMÉ ---
Attendus : 51 | Téléchargés : 51 | Échecs : 0


## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp